<center>
<h1>Welcome to the Lab 🥼🧪</h1>
</center>

## How can I analyze supply, and ongoing price change behavior of the total supply in a given market?

In this notebook, we will be looking for supply stressors that may indicate downward momentum for home prices. Specifically analyzing YoY supply trends relative to price cut changes.

#### What will you create in this notebook?

<p align="center">
  <img src="../../../images/tampa_market_price_drops.png" alt="Alt text">
</p>

#### Need help getting started?

As a reminder, you can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along.

To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY`.

Run in collab --> [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-cookbook/blob/main/examples/housing_market_research/supply_and_demand/supply_and_price_changes.ipynb)

### 1. Import the Parcl Labs Python Library

In [ ]:
# if needed, install and/or upgrade to the latest verison of the Parcl Labs Python library
%pip install --upgrade parcllabs==1.10.0 nbformat

In [2]:
import os
import pandas as pd
from parcllabs import ParclLabsClient
from parcllabs.beta.charting.styling import SIZE_CONFIG
from parcllabs.beta.charting.default_charts import create_dual_axis_chart

client = ParclLabsClient(
    api_key=os.environ.get('PARCL_LABS_API_KEY', "<your Parcl Labs API key if not set as environment variable>"), 
    limit=12 # set default limit
)

### 2. Search for Markets

In [ ]:
# in this case, lets look at Tampa market at the metro level
tampa_market = client.search.markets.retrieve(
    query='Tampa',
    sort_by='TOTAL_POPULATION',
    location_type='CITY',
    sort_order='DESC',
    limit=1
)

tampa_market.head()

In [4]:
tampa_parcl_id = tampa_market['parcl_id'].values[0]

### 3. Retrieve the Data

In [ ]:
# Lets get YoY changes for tampa market

start_date='2023-06-01'

supply = client.for_sale_market_metrics.for_sale_inventory.retrieve(
    parcl_ids=[tampa_parcl_id],
    limit=60,
    start_date=start_date
)

supply_price_changes = client.for_sale_market_metrics.for_sale_inventory_price_changes.retrieve(
    parcl_ids=[tampa_parcl_id],
    limit=60,
    start_date=start_date
)

In [ ]:
supply.head()

In [ ]:
supply_price_changes.head()

### 4. Prepare the data for analysis/charting

In [8]:
# lets focus on price drops, combine total supply with count of price drops
data = pd.merge(supply, supply_price_changes[['parcl_id', 'date', 'count_price_drop']], on=['date', 'parcl_id'], how='left')

In [9]:
data['% Price Drops'] = round(data['count_price_drop'] / data['for_sale_inventory'] * 100, 2)
data = data.rename(columns={'for_sale_inventory': 'Total Supply'})

### 5. Chart the Data

In [10]:
data = data.sort_values('date')

In [ ]:
from parcllabs.beta.charting.styling import default_style_config

default_style_config['tick_prefix'] = ''

create_dual_axis_chart(
    title='Tampa City: % of Inventory with Price Drops vs Total Supply',
    line_data=data,
    bar1_data=data,
    line_series='% Price Drops',
    bar1_series='Total Supply',
    yaxis1_title='% Price Drops',
    yaxis2_title='Total Supply',
    height=SIZE_CONFIG['x']['height'],
    width=SIZE_CONFIG['x']['width'],
    style_config=default_style_config,
    # save_path='tampa_market_price_drops.png' # to save figure
)